In [14]:
import os
base_dir = '/home/darksst/Desktop/SHAP_Project/'

In [9]:
from __future__ import print_function

import numpy as np
import warnings

from keras.models import Model
from keras.layers import Flatten, Dense, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions, preprocess_input


def VGGCustom(input_tensor=None, classes=2):
    img_rows, img_cols = 300, 300  # by default size is 224,224
    img_channels = 3

    img_dim = (img_rows, img_cols, img_channels)

    img_input = Input(shape=img_dim)

    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    # Classification block
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)

    # Create model.
    model = Model(inputs=img_input, outputs=x, name='VGG16')

    return model


# Test the function
model = VGGCustom()
print(model.summary())

Model: "VGG16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 300, 300, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 300, 300, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 150, 150, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 150, 150, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 150, 150, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 75, 75, 128)       0     

In [10]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

# Directory containing your training data
train_data_dir = os.path.join(base_dir, 'Data/tiny-imagenet-200/train')

# Data generators with augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # Using 20% of data for validation

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(300, 300),
    batch_size=1,
    class_mode='categorical',
    subset='training')  # Training data

validation_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(300, 300),
    batch_size=1,
    class_mode='categorical',
    subset='validation')  # Validation data

# Create and compile the model
model = VGGCustom(input_tensor=None, classes=200)  # Adjust the number of classes based on your dataset
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


Found 80000 images belonging to 200 classes.
Found 20000 images belonging to 200 classes.


In [11]:
# Train the model
'''history = model.fit(
    train_generator,
    epochs=1,  # Number of epochs to train
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size)'''

# Train the model
history = model.fit(
    train_generator,
    epochs=1,  # Number of epochs to train
    validation_data=validation_generator,
    steps_per_epoch=3000,
    validation_steps=1000)

3000/3000 [==============================] - 101s 33ms/step - loss: 5.3557 - accuracy: 0.0027 - val_loss: 5.3068 - val_accuracy: 0.0060


In [16]:
VGGModel = os.path.join(base_dir, 'Models/VGG16_1Feat.keras')
model.save(VGGModel, save_format='keras')